In [1]:
import torch
from transformers import BartTokenizer, BartTokenizerFast

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
version = "facebook/bart-base"
sequence = "The quick brown fox jumps over the lazy dog."
max_length = 20

# BartTokenizer

In [4]:
tokenizer: BartTokenizer = BartTokenizer.from_pretrained(version)
tokenizer

BartTokenizer(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## special ids and tokens

In [5]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 2, 3, 1, 50264]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


## tokenize(sequence) 对文本进行分词

In [5]:
tokens = tokenizer.tokenize(sequence)
tokens

['The',
 'Ġquick',
 'Ġbrown',
 'Ġfox',
 'Ġjumps',
 'Ġover',
 'Ġthe',
 'Ġlazy',
 'Ġdog',
 '.']

## convert_tokens_to_ids 将分词后的token映射为数字

In [6]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[133, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 4]

## encode = tokenize + convert_tokens_to_ids

In [7]:
print(tokenizer.encode(sequence, add_special_tokens=False))
ids = tokenizer.encode(sequence, add_special_tokens=True)
ids

[133, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 4]


[0, 133, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 4, 2]

In [8]:
# 没有开始的token
tokenizer.convert_ids_to_tokens([1])

['<pad>']

## batch_encode_plus = batch encode

In [9]:
tokenizer.batch_encode_plus([sequence], add_special_tokens=True, padding=True)

{'input_ids': [[0, 133, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## convert_ids_to_tokens 将数字映射为token

In [10]:
print(tokenizer.convert_ids_to_tokens(ids))
tokens = tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)
tokens

['<s>', 'The', 'Ġquick', 'Ġbrown', 'Ġfox', 'Ġjumps', 'Ġover', 'Ġthe', 'Ġlazy', 'Ġdog', '.', '</s>']


['The',
 'Ġquick',
 'Ġbrown',
 'Ġfox',
 'Ġjumps',
 'Ġover',
 'Ġthe',
 'Ġlazy',
 'Ġdog',
 '.']

## convert_tokens_to_string 将token转换为string

In [11]:
tokenizer.convert_tokens_to_string(tokens)

'The quick brown fox jumps over the lazy dog.'

## decode = convert_ids_to_tokens + convert_tokens_to_string

In [12]:
print(tokenizer.decode(ids))
print(tokenizer.decode(ids, skip_special_tokens=True))

<s>The quick brown fox jumps over the lazy dog.</s>
The quick brown fox jumps over the lazy dog.


## batch_decode = batch decode

In [13]:
print(tokenizer.batch_decode([ids]))
print(tokenizer.batch_decode([ids], skip_special_tokens=True))

['<s>The quick brown fox jumps over the lazy dog.</s>']
['The quick brown fox jumps over the lazy dog.']


## tokenizer([sequence])

In [14]:
inputs = tokenizer(
    sequence,  # 单个句子
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens=True,  # text添加特殊key
    return_length=True,  # 返回有效长度
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
)

print(inputs.keys())
print(inputs["input_ids"])  # 对应文字id
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 对应总长度长度

dict_keys(['input_ids', 'attention_mask', 'length'])
tensor([[    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([12])


In [15]:
inputs = tokenizer(
    [sequence] * 2,  # 句子batch
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    return_length=True,  # 返回有效长度
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 对应有效文字长度

dict_keys(['input_ids', 'length', 'attention_mask'])
tensor([[    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2],
        [    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12, 12], device='cuda:0')


# BartTokenizerFast

In [16]:
tokenizer: BartTokenizerFast = BartTokenizerFast.from_pretrained(version)
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [17]:
inputs = tokenizer(
    [sequence] * 2,  # 句子batch
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens=True,  # text添加特殊key
    return_length=True,  # 返回有效长度
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 返回文字长度时最长长度,结果不对

dict_keys(['input_ids', 'attention_mask', 'length'])
tensor([[    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2],
        [    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12, 12], device='cuda:0')
